In [509]:
"""
@author: Pablo
"""
import json
import matplotlib.pyplot as plt
import pandas
import ffmpeg
import skimage
import numpy as np
import os
import cv2
import glob

# change location of writing and reading directory to write and read files and videos
path = "D:\\Uni work\\Engineering Mathematics Work\\MDM3\Mojo\\mojo_sperm_tracking_data_bristol\\tp49\\"
os.chdir(path)

In [510]:
# open video based on the cover title and save the json file as data
with open(r"D:\Uni work\Engineering Mathematics Work\MDM3\Mojo\mojo_sperm_tracking_data_bristol\tp49\cover0_3_YOLO_NO_TRACKING_output\centroids_with_meta.json", "r") as read_file:
    data = json.load(read_file)

In [527]:
# print data with indentation for readability 
#print(json.dumps(data, sort_keys=True, indent =2))

In [512]:
#check to see if there is any non sperm classes in the video
for li in data['centroids']:
    for obj in li:
        if obj['class'] != 1:
            print(obj['class'])

In [521]:
vid = "cover0_3.avi"
path = "D:\\Uni work\\Engineering Mathematics Work\\MDM3\Mojo\\mojo_sperm_tracking_data_bristol\\tp49\\"


def playvid(path=path, vid=vid):
    vidcap =cv2.VideoCapture(path+vid)
    framerate = vidcap.get(5)
    framecount = vidcap.get(7)
#     print(framerate, framecount)
    while True:
        success, frame = vidcap.read()
        if success:
        
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            cv2.imshow('frame',gray)
            cv2.waitKey(1)
        else:
            break

    vidcap.release()
    cv2.destroyAllWindows()

    


def framesplit(path=path, vid=vid):
    os.chdir(path)
    vidcap =cv2.VideoCapture(path+vid)
    framerate = vidcap.get(5)
    framecount = vidcap.get(7)
    print(framerate, framecount)
    count = 0
    success, frame = vidcap.read()
    print(success)
    while success: 
        cv2.imwrite("frame%s.jpg" % str(count).zfill(3), frame)     # save frame as JPEG file      
        success,frame = vidcap.read()
        #print('Read a new frame: ', success)
        count += 1
    vidcap.release()
    
 
 framesplit()


In [522]:
#loop through all sperms to find their bounding box and center points for plotting
def getbboxes():
    frame_bboxes=[]
    frame_centroids= []
    frame_ids = []
    for li in data['centroids']: 
        bboxes = []
        centers = []
        ids = [] 
        for obj in li:
            bboxes.append(obj['bbox'])
            centers.append(obj['center'])
            ids.append(obj['id'])
        frame_bboxes.append(bboxes)
        frame_centroids.append(centers)
        frame_ids.append(ids)
    return frame_bboxes, frame_centroids, frame_ids



In [515]:
bboxes, centroids, ids = getbboxes()
def annotateframe(frame=0, showframe=False, saveimage=True, bboxes=bboxes, centroids=centroids, ids=ids):
    frame=frame
    resizefactor = 2
    img = cv2.imread('frame{}.jpg'.format(str(frame).zfill(3)))
    # imgray = cv2.cvtColor(imgresized,cv2.COLOR_BGR2GRAY)
    
    for box in bboxes[frame]:
        #define bounding box coordinates
        x_min, y_min, w, h = resizefactor*int(box[0]),resizefactor*int(box[1]),resizefactor*int(box[2]),resizefactor*int(box[3])
        #print(x_min, y_min, w, h)
        startpoint = (x_min, y_min)
        endpoint =  (x_min+w, y_min+h)
        # draw a green rectangle to visualize the bounding rect
        img=cv2.rectangle(img,startpoint,endpoint, (0, 255, 0), 2)
        
    for count, centroid in enumerate(centroids[frame]):
        x, y = resizefactor* int(centroid[0]), resizefactor* int(centroid[1])
        # draw a point at each sperm
        img = cv2.circle(img, (x,y), radius=3, color=(0, 0, 255), thickness=-1)
        
        # label each sperm
        img =cv2.putText(img, "#{}".format(ids[frame][count]), (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,0,0), 2)
    #display frame in a pop up
    if showframe == True:
        cv2.imshow('output',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

#     #save annotated image
    if saveimage is True:
        cv2.imwrite("frame{}.jpg".format(str(frame).zfill(3)),img)  


In [516]:
# annotate all frames in a video, note that some videos have one or two frames less 
for i in range(0,301):
    annotateframe(frame=i, showframe=False)

In [525]:
def makevideo(vid=vid):
    img_array = []
    for filename in glob.glob(r'D:\Uni work\Engineering Mathematics Work\MDM3\Mojo\mojo_sperm_tracking_data_bristol\tp49\*.jpg'):
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
   
    out = cv2.VideoWriter('annotated{}.avi'.format(vid),cv2.VideoWriter_fourcc(*'DIVX'), 60, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()

In [526]:
makevideo(vid=vid)